<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Create-the-Schema-and-dataset" data-toc-modified-id="Create-the-Schema-and-dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create the Schema and dataset</a></span><ul class="toc-item"><li><span><a href="#Field-Group-Creation" data-toc-modified-id="Field-Group-Creation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Field Group Creation</a></span></li><li><span><a href="#Schema-creation" data-toc-modified-id="Schema-creation-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Schema creation</a></span></li><li><span><a href="#Change-the-Schema-Timezone-reference." data-toc-modified-id="Change-the-Schema-Timezone-reference.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Change the Schema Timezone reference.</a></span></li><li><span><a href="#Dataset-Creation" data-toc-modified-id="Dataset-Creation-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Dataset Creation</a></span></li></ul></li><li><span><a href="#Prepare-for-AEP-data-Ingestion" data-toc-modified-id="Prepare-for-AEP-data-Ingestion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prepare for AEP data Ingestion</a></span></li><li><span><a href="#Data-Ingestion" data-toc-modified-id="Data-Ingestion-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Ingestion</a></span></li><li><span><a href="#CJA-Connection-and-DataView" data-toc-modified-id="CJA-Connection-and-DataView-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>CJA Connection and DataView</a></span></li></ul></div>

In this notebook, we will use the power of aepp to load the data in the AEP data lake and then use that data in CJA as summary data.\
Due to time constraint, the data has been pre-loaded in CJA for this lab.

# Setup

In [1]:
import aepp
import pandas as pd
from datetime import datetime
import uuid, json

# Load the data

From the previous notebook, we have created a file name `'prediction_orders.2024.2025.csv'`.\
We will load the data associated with this file using pandas and change some elements in order to better represent the information we will want to load into AEP. 

In [ ]:
df = pd.read_csv('prediction_orders.2024.2025.csv',sep=';')

In [ ]:
df['Orders'].max()

We want the order to be sent as Integer and not float, as there is no half order in real life.

In [ ]:
df['Orders'] = df['Orders'].astype(int)

In [ ]:
df.head()

# Create the Schema and dataset

We will connect to AEP API via `aepp` in order to create the different element needed to load the data in AEP.

In [2]:
prod = aepp.importConfigFile('myconfig.json',sandbox='bighouse',connectInstance=True)

From aepp, we will need to use the following modules:
* fieldgroupmanager : To easily create a field group with the different field names
* schemamanager : To easily create a schema
* catalog : To create a dataset to load the data into.
* ingestion : To create the batch to load the into the dataset.
* som : To easily create the objects that we want to load via the ingestion module.

In [3]:
from aepp import ingestion, catalog, schemamanager, fieldgroupmanager, som

## Field Group Creation

A field group is a definition of different fields that are usually combine together in a business related view.\
Most of the time, the users are creating the field groups via the UI but in our example, we don't want to be pull away from our notebook, so we will use the power of aepp to generate the fields in the new field group to load the data.\
Field Groups are always associated to a class in order to limit their footprint, in our case, we will use the class defining the  summary metrics

In [4]:
predictionFieldGroup = fieldgroupmanager.FieldGroupManager(title='prediction',fg_class=['https://ns.adobe.com/xdm/classes/summarymetrics'],config=prod)

Checking the current representation

In [5]:
predictionFieldGroup.to_dict()

{'_acxpevangelist': {}}

We will create a node, called `prediction` and we will add the number of orders predicted in that node as an integer (`predictedOrders`) 

In [6]:
predictionFieldGroup.addField('_acxpevangelist.prediction',dataType='object',objectComponents={'predictedOrders':'integer'})

{'title': 'prediction',
 'meta:resourceType': 'mixins',
 'description': 'powered by aepp',
 'type': 'object',
 'definitions': {'customFields': {'type': 'object',
   'properties': {'_acxpevangelist': {'properties': {'prediction': {'type': 'object',
       'title': 'prediction',
       'description': '',
       'properties': {'predictedOrders': {'type': 'integer'}}}},
     'type': 'object'}}},
  'property': {'type': 'object',
   'properties': {'_acxpevangelist': {'properties': {}, 'type': 'object'}}}},
 'allOf': [{'$ref': '#/definitions/customFields', 'type': 'object'},
  {'$ref': '#/definitions/property', 'type': 'object'}],
 'meta:intendedToExtend': ['https://ns.adobe.com/xdm/classes/summarymetrics'],
 'meta:containerId': 'tenant',
 'meta:tenantNamespace': '_acxpevangelist'}

You can see the result of the element being added to the Field Group.\
Below a more visual representation:

In [7]:
predictionFieldGroup.to_dict()

{'_acxpevangelist': {'prediction': {'predictedOrders': 'integer'}}}

In order to create that field group, we can just use the `createFieldGroup` method

In [12]:
predictionFieldGroup.createFieldGroup()

{'$id': 'https://ns.adobe.com/acxpevangelist/mixins/a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c',
 'meta:altId': '_acxpevangelist.mixins.a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c',
 'meta:resourceType': 'mixins',
 'version': '1.0',
 'title': 'prediction',
 'type': 'object',
 'description': 'powered by aepp',
 'definitions': {'customFields': {'type': 'object',
   'properties': {'_acxpevangelist': {'properties': {'prediction': {'type': 'object',
       'title': 'prediction',
       'description': '',
       'properties': {'predictedOrders': {'type': 'integer',
         'minimum': -2147483648,
         'maximum': 2147483647,
         'meta:xdmType': 'int'}},
       'meta:xdmType': 'object'}},
     'type': 'object',
     'meta:xdmType': 'object'}},
   'meta:xdmType': 'object'},
  'property': {'type': 'object',
   'properties': {'_acxpevangelist': {'properties': {},
     'type': 'object',
     'meta:xdmType': 'object'}},
   'meta:xdmType': 'object'}},
 'allOf': [{'$ref': '#/defi

## Schema creation

A schema is composed of a class and field groups.\
We will use the Field Group created before to create the schema.\
The class used will be the `'https://ns.adobe.com/xdm/classes/summarymetrics'` class that represent the summary data. 

In [8]:
cjaSummaryPrediction = schemamanager.SchemaManager(title='AdobeStore Predictions',schemaClass='https://ns.adobe.com/xdm/classes/summarymetrics',config=prod)

We simply needs to add the field group previously created to that schema

In [9]:
cjaSummaryPrediction.addFieldGroup('https://ns.adobe.com/acxpevangelist/mixins/a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c')

{
  "$id": "https://ns.adobe.com/acxpevangelist/mixins/a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c",
  "meta:altId": "_acxpevangelist.mixins.a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c",
  "meta:resourceType": "mixins",
  "version": "1.0",
  "title": "prediction",
  "type": "object",
  "description": "powered by aepp",
  "definitions": {
    "customFields": {
      "type": "object",
      "properties": {
        "_acxpevangelist": {
          "properties": {
            "prediction": {
              "type": "object",
              "title": "prediction",
              "description": "",
              "properties": {
                "predictedOrders": {
                  "type": "integer",
                  "minimum": -2147483648,
                  "maximum": 2147483647,
                  "meta:xdmType": "int"
                }
              },
              "meta:xdmType": "object"
            }
          },
          "type": "object",
          "meta:xdmType": "object"
      

We can verify that the schema is looking as expected via the `to_dict()` method.

In [10]:
cjaSummaryPrediction.to_dict()

{'_id': 'string',
 'eventType': 'string',
 'timestamp': 'string',
 '_acxpevangelist': {'prediction': {'predictedOrders': 'integer'}}}

We can then directly create the schema from this `SchemaManager` class.

In [16]:
cjaSummaryPrediction.createSchema()

{'$id': 'https://ns.adobe.com/acxpevangelist/schemas/1d8f0ad11daa2fece55a19b76283272d0cf6980205103332',
 'meta:altId': '_acxpevangelist.schemas.1d8f0ad11daa2fece55a19b76283272d0cf6980205103332',
 'meta:resourceType': 'schemas',
 'version': '1.0',
 'title': 'AdobeStore Predictions',
 'type': 'object',
 'description': 'powered by aepp',
 'allOf': [{'$ref': 'https://ns.adobe.com/xdm/classes/summarymetrics',
   'type': 'object',
   'meta:xdmType': 'object'},
  {'$ref': 'https://ns.adobe.com/acxpevangelist/mixins/a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c',
   'type': 'object',
   'meta:xdmType': 'object'}],
 'refs': ['https://ns.adobe.com/acxpevangelist/mixins/a457146466ee18ccc1f109d8a128ae163d3560948a8fca1c',
  'https://ns.adobe.com/xdm/classes/summarymetrics'],
 'required': ['xdm:timestamp'],
 'imsOrg': 'D0F83C645C5E1CC60A495CB3@AdobeOrg',
 'additionalInfo': {'numberOfIdentities': 0,
  'numberOfRelationShips': 0,
  'classTitle': 'XDM Summary Metrics',
  'hasRelationShip': False,
  

## Change the Schema Timezone reference.

By default, the timezone associated with your data in the `SummaryData` class will be based of UTC timezone.\
However, you may have your Data View setup with a different timezone.\
In that case, you will need to do an operation with the AEP API on the schema.\
This is where `aepp` and the `schemamanager` module and `SchemaManager` class come handy.\
We will use the modules to create a `descriptor`, which is then attached the schema itself.\
A descriptor is a meta data information that is attached to a schema. 

In the next cells, we will take the ID of the schema to instantiate a new SchemaManager class, however, you could directly do this operation after the creation of the schema itself.

In [11]:
cjaSummaryPrediction = schemamanager.SchemaManager('https://ns.adobe.com/acxpevangelist/schemas/1d8f0ad11daa2fece55a19b76283272d0cf6980205103332',config=prod)

You can check which descriptor exist by using the `DESCRIPTOR_TYPES` attributes.\
The one we are interested in is `'xdm:descriptorTimeSeriesGranularity'`

In [12]:
cjaSummaryPrediction.DESCRIPTOR_TYPES

['xdm:descriptorIdentity',
 'xdm:alternateDisplayInfo',
 'xdm:descriptorOneToOne',
 'xdm:descriptorReferenceIdentity',
 'xdm:descriptorDeprecated',
 'xdm:descriptorTimeSeriesGranularity']

In [13]:
mydescriptor = cjaSummaryPrediction.createDescriptorOperation('xdm:descriptorTimeSeriesGranularity',timezone='America/Los_Angeles',granularity='day')

In [14]:
mydescriptor

{'@type': 'xdm:descriptorTimeSeriesGranularity',
 'xdm:sourceSchema': 'https://ns.adobe.com/acxpevangelist/schemas/1d8f0ad11daa2fece55a19b76283272d0cf6980205103332',
 'xdm:sourceVersion': 1,
 'xdm:granularity': 'day',
 'xdm:ianaTimezone': 'America/Los_Angeles'}

You can then use that new definition store in that variable to add this descriptor to the schema. 

In [15]:
cjaSummaryPrediction.createDescriptor(mydescriptor)

{'@id': '183214a37781aaf0833cca3a04903e684c2caa81b1138c39',
 '@type': 'xdm:descriptorTimeSeriesGranularity',
 'xdm:sourceSchema': 'https://ns.adobe.com/acxpevangelist/schemas/1d8f0ad11daa2fece55a19b76283272d0cf6980205103332',
 'xdm:sourceVersion': 1,
 'imsOrg': 'D0F83C645C5E1CC60A495CB3@AdobeOrg',
 'version': '1',
 'xdm:granularity': 'day',
 'xdm:ianaTimezone': 'America/Los_Angeles',
 'meta:containerId': '1f7f1c02-b926-432b-bf1c-02b926a32bc2',
 'meta:sandboxId': '1f7f1c02-b926-432b-bf1c-02b926a32bc2',
 'meta:sandboxType': 'production'}

## Dataset Creation

Once you have create the schema, you can then create a dataset directly.\
As usual, this task can be done directly via the UI.\
In our notebook, we will keep going to use the API to keep our flow going.

The dataset can be created via the Catalog API, that is part of the `catalog` module.

In [16]:
cat = catalog.Catalog(config=prod)

To create a dataset, you can use the parameterize version of the method in order to simplify your task.

In [18]:
cat.createDataSets(name='Summary Data - CJA Summit 2025 lab123',
                   schemaId='https://ns.adobe.com/acxpevangelist/schemas/1d8f0ad11daa2fece55a19b76283272d0cf6980205103332')

['@/dataSets/67aba67f109a4f2aee087796']

You can save that dataset ID, as it will be useful for the future on data ingestion 

In [17]:
datasetId = '67aba67f109a4f2aee087796'

# Prepare for AEP data Ingestion

With the data that has been loaded previously from your csv, you can go through the different rows and load the data in the different payload you would like to have.\
Our organization is `_acxpevangelist` and we can use it at the start before creating the different payload when reading the data.\
The `Som` class is an easy way to build complex data structure without caring of nested struct.

In [ ]:
dataToLoad = [] ## the complete payload that will have all the different dates
for index, row in df.iterrows():
    mysom = som.Som() ## initatiate with the Som
    date = datetime.fromisoformat(row['Date']) ## reading the date
    timestamp = date.replace(hour=6).timestamp() ## setting the timestamp 
    mysom.assign('_id',str(uuid.uuid4())) ## setting a uuid
    mysom.assign('timestamp',int(timestamp*1000)) ## setting the timestamp in milliseconds
    mysom.assign('eventType','predictionData')
    mysom.assign('_acxpevangelist.prediction.predictedOrders',row['Orders'])
    dataToLoad.append(mysom.to_dict())

Verifying the data

In [ ]:
dataToLoad[:5]

Verifying the timestamp with `time` module for the first iteration:

In [ ]:
import time
print(time.ctime(1735707600000/1000)) ## only support seconds
print(time.ctime(1736053200000/1000)) ## only support seconds

You can then save the data in a JSON format if you wish to send it via different form (directly in the UI or via Source).

In [ ]:
with open('predicted_data.json','w') as f:
    json.dump(dataToLoad,f,indent=2)

# Data Ingestion

As usual, we will use the Notebook to complete our test by ingesting the data into AEP.\
We will use the `ingestion` module and directly load a small batch to AEP via the notebook.

In [13]:
dataIngestion = ingestion.DataIngestion(config=prod)

The process to load data via a batch in AEP is in 3 steps:
1. Create a Batch to receive data
2. Load the data in the batch (using the Batch ID created in the first operation)
3. Finish the batch by sending the `COMPLETE` operation.

In [15]:
mybatch = dataIngestion.createBatch(datasetId=datasetId,format="json",multiline=True)

In [16]:
mybatch

{'id': '01JM04MH5QAEJQK4Q83EH1TS68',
 'imsOrg': 'D0F83C645C5E1CC60A495CB3@AdobeOrg',
 'updated': 1739466622372,
 'status': 'loading',
 'created': 1739466622372,
 'relatedObjects': [{'type': 'dataSet', 'id': '67aba67f109a4f2aee087796'}],
 'version': '1.0.0',
 'tags': {'acp_stagePath': ['acp_foundation_push/stage/01JM04MH5QAEJQK4Q83EH1TS68'],
  'acp_requestType': ['user'],
  'acp_dataSetViewId': ['67aba681109a4f2aee087798'],
  'acp_type': ['ingest'],
  'acp_producer': ['fbb6f6a5024e45d08b5f346a3c9a540b',
   'aep/siphon/bi/uploadMode::']},
 'createdUser': '231A1E846763328F0A495FB7@techacct.adobe.com',
 'updatedUser': '231A1E846763328F0A495FB7@techacct.adobe.com',
 'externalId': '01JM04MH5QAEJQK4Q83EH1TS68',
 'createdClient': 'acp_foundation_push',
 'inputFormat': {'format': 'json', 'isMultiLineJson': True}}

In [17]:
res = dataIngestion.uploadSmallFile(batchId=mybatch['id'],datasetId=datasetId,filePath='data.prediction.json',data=dataToLoad)
res

{}

No error are returned, we can close the batch

In [18]:
dataIngestion.uploadSmallFileFinish(batchId=mybatch['id'])

{}

Once your batch is ingested, you can also monitor its status via the Catalog API.\
It will turn as `success` if everything is fine.

In [21]:
cat.getBatch(mybatch['id'])

{'01JM04MH5QAEJQK4Q83EH1TS68': {'status': 'staging',
  'tags': {'acp_stagePath': ['acp_foundation_push/stage/01JM04MH5QAEJQK4Q83EH1TS68'],
   'acp_sloPolicyName': ['live10Mb'],
   'acp_workflow': ['ValveWorkflow'],
   'acp_requestType': ['user'],
   'acp_latencyTargetInMillis': ['300000'],
   'acp_dataSetViewId': ['67aba681109a4f2aee087798'],
   'acp_type': ['ingest'],
   'siphon/valve/stage/ingest': ['{"id":"98049a059066441f8509a9ee7b11df5e","status":"created","createdAt":1739466646570,"batchId":"01JM04MH5QAEJQK4Q83EH1TS68","imsOrg":"D0F83C645C5E1CC60A495CB3@AdobeOrg","bulkHead":"live","service":"platform.siphon.ingest","properties":{}}'],
   'acp_bulkHead': ['live'],
   'acp_producer': ['fbb6f6a5024e45d08b5f346a3c9a540b',
    'aep/siphon/bi/uploadMode::'],
   'acp_requestId': ['eFDRu4lQnuIeNYLm4vaC9m3jLGQJQZTy'],
   'acp_buffered': ['false'],
   'acp_latencyMaxInMillis': ['10800000']},
  'relatedObjects': [{'type': 'dataSet', 'id': '67aba67f109a4f2aee087796'}],
  'id': '01JM04MH5QAEJ

# CJA Connection and DataView

As of today, once the data has been ingested in AEP, CJA does not provide an API to load this new dataset into the existing connection, neither a way to programmatically update the Data View with these new components.\
These operations have been done manually via the UI prior to this Lab.\
I hope you could still the possibility of automation and improvement in your workflow via this Lab.

You can see the result of the upload directly in a [CJA Workspace](https://experience.adobe.com/#/@acxpevangelist/platform/analytics/#/workspace/edit/67805d4f6d2b093a21038293)

Do not forget to vote for the Lab and support via any mean the repos for `cjapy` and `aepp`, both open source solutions. 